In [1]:
import numpy as np 
import pandas as pd 
import os

from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [2]:
# Чтение данных
csv = spark.read.csv('flights.csv', inferSchema=True, header=True)
csv.show(10)

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
|        19|        5|     DL|          10397|        15016|      -1|     -19|
|        19|        5|     DL|          15016|        10397|       0|      -1|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|      

In [3]:
# Выбор нужных столбцов и создание метки для регрессии
data1 = csv.select("DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay", "ArrDelay")

In [4]:
# Создание признакового вектора
assembler = VectorAssembler(inputCols=["DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay"], outputCol="features")
data1 = assembler.transform(data1)

In [5]:
# Разделение данных на обучающий и тестовый наборы
splits = data1.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]

In [6]:
# Создание и обучение модели RandomForestRegressor
rf = RandomForestRegressor(featuresCol="features", labelCol="ArrDelay")
pipeline_rf = Pipeline(stages=[rf])

# Создание сетки параметров для подбора
paramGrid_rf = ParamGridBuilder().addGrid(rf.maxDepth, [5, 10]).build()

# Оценщик для регрессии
evaluator_rf = RegressionEvaluator(labelCol="ArrDelay")

In [7]:
# Кросс-валидация с подбором параметров
crossval_rf = CrossValidator(estimator=pipeline_rf,
                             estimatorParamMaps=paramGrid_rf,
                             evaluator=evaluator_rf,
                             numFolds=3)

In [8]:
# Обучение модели случайного леса
model_rf = crossval_rf.fit(train)

In [9]:
# Прогнозирование на тестовом наборе
predictions_rf = model_rf.transform(test)

# Оценка результатов
predictions_rf.show(10)

+----------+---------+---------------+-------------+--------+--------+--------------------+-------------------+
|DayofMonth|DayOfWeek|OriginAirportID|DestAirportID|DepDelay|ArrDelay|            features|         prediction|
+----------+---------+---------------+-------------+--------+--------+--------------------+-------------------+
|         1|        1|          10140|        10397|      -4|     -11|[1.0,1.0,10140.0,...|-7.1845420566431715|
|         1|        1|          10140|        11259|      21|      19|[1.0,1.0,10140.0,...| 17.239963791496614|
|         1|        1|          10140|        11292|      -4|      -8|[1.0,1.0,10140.0,...|  -7.67980527529289|
|         1|        1|          10140|        11292|      -1|      -6|[1.0,1.0,10140.0,...| -6.472043296648529|
|         1|        1|          10140|        11298|     -10|     -13|[1.0,1.0,10140.0,...|-11.469244153319925|
|         1|        1|          10140|        11298|      -6|     -25|[1.0,1.0,10140.0,...| -9.598097065

In [10]:
# Оценка результатов
rmse_rf = evaluator_rf.evaluate(predictions_rf, {evaluator_rf.metricName: "rmse"})
print(f"Root Mean Squared Error (Random Forest): {rmse_rf}")

Root Mean Squared Error (Random Forest): 19.65235649624361


In [11]:
# Выбор нужных столбцов и создание метки для бинарной классификации
data2 = csv.select("DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay", ((col("ArrDelay") > 15).cast("Int").alias("label")))


In [12]:
# Создание признакового вектора
assembler = VectorAssembler(inputCols=["DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay"], outputCol="features")
data2 = assembler.transform(data2)


In [13]:
# Разделение данных на обучающий и тестовый наборы
splits = data2.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]

In [14]:
# Создание и обучение модели Logistic Regression
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=10, regParam=0.3, elasticNetParam=0.8)
pipeline_lr = Pipeline(stages=[lr])

# Создание сетки параметров для подбора
paramGrid_lr = ParamGridBuilder().addGrid(lr.regParam, [0.1, 0.01]).build()

# Оценщик для бинарной классификации
evaluator_lr = BinaryClassificationEvaluator(labelCol="label")

In [15]:
# Кросс-валидация с подбором параметров
crossval_lr = CrossValidator(estimator=pipeline_lr,
                             estimatorParamMaps=paramGrid_lr,
                             evaluator=evaluator_lr,
                             numFolds=3)

In [16]:
# Обучение модели логистической регрессии
model_lr = crossval_lr.fit(train)

In [17]:
# Прогнозирование на тестовом наборе
predictions_lr = model_lr.transform(test)

# Оценка результатов
predictions_lr.show(10)

# Отображение только строк с предсказанием 1
predictions_lr.filter(predictions_lr["prediction"] == 1).show(10)

+----------+---------+---------------+-------------+--------+-----+--------------------+--------------------+--------------------+----------+
|DayofMonth|DayOfWeek|OriginAirportID|DestAirportID|DepDelay|label|            features|       rawPrediction|         probability|prediction|
+----------+---------+---------------+-------------+--------+-----+--------------------+--------------------+--------------------+----------+
|         1|        1|          10140|        10397|      -4|    0|[1.0,1.0,10140.0,...|[1.86770426424062...|[0.86619241816556...|       0.0|
|         1|        1|          10140|        10397|      -2|    0|[1.0,1.0,10140.0,...|[1.81333461358252...|[0.85976440972877...|       0.0|
|         1|        1|          10140|        10821|       8|    0|[1.0,1.0,10140.0,...|[1.54148636029204...|[0.82368069439152...|       0.0|
|         1|        1|          10140|        11259|      -3|    0|[1.0,1.0,10140.0,...|[1.84051943891157...|[0.86301012896704...|       0.0|
|     

In [18]:
# Оценка результатов
areaUnderROC_lr = evaluator_lr.evaluate(predictions_lr)
print(f"Area Under ROC (Logistic Regression): {areaUnderROC_lr}")

Area Under ROC (Logistic Regression): 0.9228066991369412
